In [1]:
import os
import time
import requests

ELEFAN_PRODUCT_API_URL = os.environ.get("API_URL")
OFF_API_URL = "https://fr.openproductsfacts.org/api/v2/product"
OFF_KEY = "OPF"

ELEFAN_PRODUCTS_FAMILLE_CODE_TO_INCLUDE = [
    24,  # cuisine
    31,  # journaux
    41,  # maison
    42,  # papeterie
    43,  # textile
    44,  # animalerie
    45,  # jardinerie
    90,  # jeux
]

In [2]:
"""
Récupérer les produits depuis Elefan (via l'API)
"""
print("Elefan Grenoble")
elefan_product_list = list()

response = requests.get(ELEFAN_PRODUCT_API_URL)
elefan_product_list = response.json()
print("Nombre d'articles (dans KASO) :", len(elefan_product_list))
# print(elefan_product_list[0])

elefan_product_actifs_list = [p for p in elefan_product_list if p['status'] == 'ACTIF']
print("Nombre d'articles actifs (status = ACTIF dans KASO) :", len(elefan_product_actifs_list))

elefan_product_actifs_ean_list = [p for p in elefan_product_actifs_list if (len(str(p['code'])) in [8, 13])]
print("Nombre d'articles actifs avec un code-barres EAN (8 ou 13 chiffres) :", len(elefan_product_actifs_ean_list))

elefan_product_actifs_ean_filtered_list = [p for p in elefan_product_actifs_ean_list if p['famille']['code'] in ELEFAN_PRODUCTS_FAMILLE_CODE_TO_INCLUDE]
print("Nombre d'articles actifs avec un code-barres EAN et filtrés (catégories) :", len(elefan_product_actifs_ean_filtered_list))

Elefan Grenoble
Nombre d'articles (dans KASO) : 2106
Nombre d'articles actifs (status = ACTIF dans KASO) : 1456
Nombre d'articles actifs avec un code-barres EAN (8 ou 13 chiffres) : 1124
Nombre d'articles actifs avec un code-barres EAN et filtrés (catégories) : 189


In [3]:
print(f"Lien avec {OFF_KEY} (pour les {len(elefan_product_actifs_ean_filtered_list)} articles filtrés)")

found = 0
for index, elefan_product in enumerate(elefan_product_actifs_ean_filtered_list):
    response = requests.get(f"{OFF_API_URL}/{elefan_product['code']}.json")
    if response.status_code == 200:
        found += 1
        elefan_product_actifs_ean_filtered_list[index][OFF_KEY] = response.json()
    if (index % 10) == 0:
        time.sleep(1)
    if (index % 50) == 0:
        print(f"progress: {index}...")

print("Trouvés", found, "/", len(elefan_product_actifs_ean_filtered_list))

Lien avec OPF (pour les 189 articles filtrés)
progress: 0...
progress: 50...
progress: 100...
progress: 150...
Trouvés 2 / 189


In [4]:
elefan_product_actifs_ean_filtered_found_list = [p for p in elefan_product_actifs_ean_filtered_list if OFF_KEY in p]
print(f"Sur les {len(elefan_product_actifs_ean_filtered_found_list)} produits trouvés dans {OFF_KEY}...")

for product in elefan_product_actifs_ean_filtered_found_list:
    print({key: product[key] for key in ["code", "designation"]})

Sur les 2 produits trouvés dans fOPF...
{'code': 40267654, 'designation': 'COLLE BATON Ť UHU ť'}
{'code': 3760147600064, 'designation': 'JOUECABOIS 200 PCS'}
